# Overview
- nb001で出力したデータをSVC、ランダムフォレストのグリッドサーチで解析。

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 100)

import seaborn as sns
import string
from sklearn.utils import shuffle
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

## 1) データ読み込み

In [6]:
data_train = pd.read_csv('../data/train_nb001.csv', index_col=0)
data_train.tail()

,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_ind_02_cat_2,ps_ind_02_cat_3,ps_ind_02_cat_4,ps_ind_04_cat_1,ps_ind_05_cat_1,ps_ind_05_cat_2,ps_ind_05_cat_3,ps_ind_05_cat_4,ps_ind_05_cat_6,ps_car_01_cat_4,ps_car_01_cat_5,...,ps_reg_01 ps_car_15,ps_reg_01 ps_calc_01,ps_reg_01 ps_calc_02,ps_reg_01 ps_calc_03,ps_reg_02^2,ps_reg_02 ps_reg_03,ps_reg_02 ps_car_12,ps_reg_02 ps_car_13,ps_reg_02 ps_car_14,ps_reg_02 ps_car_15,ps_reg_02 ps_calc_01,ps_reg_02 ps_calc_02,ps_reg_02 ps_calc_03,ps_reg_03^2,ps_reg_03 ps_car_12,ps_reg_03 ps_car_13,ps_reg_03 ps_car_14,ps_reg_03 ps_car_15,ps_reg_03 ps_calc_01,ps_reg_03 ps_calc_02,ps_reg_03 ps_calc_03,ps_car_12^2,ps_car_12 ps_car_13,ps_car_12 ps_car_14,ps_car_12 ps_car_15,ps_car_12 ps_calc_01,ps_car_12 ps_calc_02,ps_car_12 ps_calc_03,ps_car_13^2,ps_car_13 ps_car_14,ps_car_13 ps_car_15,ps_car_13 ps_calc_01,ps_car_13 ps_calc_02,ps_car_13 ps_calc_03,ps_car_14^2,ps_car_14 ps_car_15,ps_car_14 ps_calc_01,ps_car_14 ps_calc_02,ps_car_14 ps_calc_03,ps_car_15^2,ps_car_15 ps_calc_01,ps_car_15 ps_calc_02,ps_car_15 ps_calc_03,ps_calc_01^2,ps_calc_01 ps_calc_02,ps_calc_01 ps_calc_03,ps_calc_02^2,ps_calc_02 ps_calc_03,ps_calc_03^2,target
216935,5,7,0,0,1,0,0,8,0,1,0,0.6,0.4,1.537652,1,0.424264,1.269111,0.384708,3.162278,0.5,0.1,0.5,3,4,9,1,10,2,9,10,1,5,7,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,1.897367,0.30,0.06,0.30,0.16,0.615061,0.169706,0.507645,0.153883,1.264911,0.20,0.04,0.20,2.364375,0.652371,1.951452,0.591547,4.862484,0.768826,0.153765,0.768826,0.18,0.538438,0.163218,1.341641,0.212132,0.042426,0.212132,1.610644,0.488237,4.013282,0.634556,0.126911,0.634556,0.1480,1.216553,0.192354,0.038471,0.192354,10.0,1.581139,0.316228,1.581139,0.25,0.05,0.25,0.01,0.05,0.25,1
216936,1,5,0,0,1,0,0,4,0,0,1,0.3,0.4,0.897653,2,0.424264,0.756979,0.400000,2.000000,0.3,0.4,0.6,1,1,8,2,9,5,9,9,2,1,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.600000,0.09,0.12,0.18,0.16,0.359061,0.169706,0.302791,0.160000,0.800000,0.12,0.16,0.24,0.805782,0.380842,0.679504,0.359061,1.795307,0.269296,0.359061,0.538592,0.18,0.321159,0.169706,0.848528,0.127279,0.169706,0.254558,0.573017,0.302791,1.513957,0.227094,0.302791,0.454187,0.1600,0.800000,0.120000,0.160000,0.240000,4.0,0.600000,0.800000,1.200000,0.09,0.12,0.18,0.16,0.24,0.36,1
216937,0,6,0,1,0,0,0,9,0,1,0,0.9,0.3,0.711952,3,0.400000,0.970654,0.372424,3.464102,0.5,0.4,0.6,2,2,8,2,8,4,10,3,2,1,6,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,3.117691,0.45,0.36,0.54,0.09,0.213585,0.120000,0.291196,0.111727,1.039230,0.15,0.12,0.18,0.506875,0.284781,0.691059,0.265148,2.466272,0.355976,0.284781,0.427171,0.16,0.388262,0.148970,1.385641,0.200000,0.160000,0.240000,0.942169,0.361495,3.362445,0.485327,0.388262,0.582392,0.1387,1.290116,0.186212,0.148970,0.223455,12.0,1.732051,1.385641,2.078461,0.25,0.20,0.30,0.16,0.24,0.36,1
216938,0,2,1,0,0,0,0,2,0,0,1,0.6,0.1,0.577170,2,0.316228,0.876295,0.320780,3.741657,0.5,0.2,0.6,3,0,9,3,10,4,11,5,0,4,4,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,...,2.244994,0.30,0.12,0.36,0.01,0.057717,0.031623,0.087629,0.032078,0.374166,0.05,0.02,0.06,0.333125,0.182517,0.505771,0.185145,2.159572,0.288585,0.115434,0.346302,0.10,0.277109,0.101440,1.183216,0.158114,0.063246,0.189737,0.767893,0.281098,3.278795,0.438147,0.175259,0.525777,0.1029,1.200250,0.160390,0.064156,0.192468,14.0,1.870829,0.748331,2.244994,0.25,0.10,0.30,0.04,0.12,0.36,1
216939,1,1,0,0,1,0,0,13,1,0,0,0.6,0.4,1.095160,2,0.374166,0.752558,0.328634,3.464102,0.2,0.6,0.3,1,1,8,2,11,0,13,7,2,4,9,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,2.078461,0.12,0.36,0.18,0.16,0.438064,0.149666,0.301023,0.131453,1.385641,0.08,0.24,0.12,1.199375,0.4

In [7]:
X = data_train.drop('target', axis=1)
y = data_train['target']

## 2) 解析

In [8]:
# 訓練用、CV用にデータ分割
# -------------------------
X_train, X_check, y_train, y_check = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)   # 訓練:テスト = 80:20

print('Label counts in y_train: [0 1] =', np.bincount(y_train.astype(np.int64)))
print('Label counts in y_check: [0 1] =', np.bincount(y_check.astype(np.int64)))

Label counts in y_train: [0 1] = [156197  17355]
Label counts in y_check: [0 1] = [39049  4339]


In [26]:
# =================================
# Pipeline: pl_svc
# SVC / k分割交差検証 / グリッドサーチ
# =================================

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

### max_iter ###
pl_svc = make_pipeline(StandardScaler(), SVC(random_state=21, max_iter=100))

from sklearn.model_selection import GridSearchCV

### Grid Search試行用 ###
#svc_param_range = [0.01, 0.1]
#svc_param_grid = [{'svc__C': svc_param_range, 'svc__kernel': ['linear']},
#                  {'svc__C': svc_param_range, 'svc__kernel': ['poly', 'rbf', 'sigmoid'], 'svc__gamma': svc_param_range}]

### Grid Search本番用 ###
svc_param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
svc_param_grid = [{'svc__C': svc_param_range, 'svc__kernel': ['linear']},
                  {'svc__C': svc_param_range, 'svc__kernel': ['poly', 'rbf', 'sigmoid'], 'svc__gamma': svc_param_range}]

svc_gs = GridSearchCV(estimator=pl_svc, param_grid=svc_param_grid, scoring='accuracy', cv=10, refit=True, n_jobs=-1)
svc_gs.fit(X_train, y_train)

print('CV best accuracy:', svc_gs.best_score_)
print('Best parameters:', svc_gs.best_params_)
svc_bestclf = svc_gs.best_estimator_
print('Test accuracy: %f' % svc_bestclf.score(X_check, y_check))

#svc_pred = svc_bestclf.predict(X_test)

C:\Users\ultra\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


CV best accuracy: 0.8970395156417021
Best parameters: {'svc__C': 1.0, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}
Test accuracy: 0.895755


In [9]:
# =============================================
# Pipeline: pl_randf
# ランダムフォレスト / k分割交差検証 / グリッドサーチ
# =============================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pl_randf = make_pipeline(RandomForestClassifier(random_state=21, n_jobs=-1))
34
from sklearn.model_selection import GridSearchCV

### Grid Search試行用 ###
rf_param_estimators_range = [100, 200]
rf_param_depth_range = [5, 10]
rf_param_split_range = [5, 10]

### Grid Search本番用 ###
#rf_param_estimators_range = [100, 200, 300, 400, 500]
#rf_param_depth_range = [5, 10, 15, 20, 25, 30]
#rf_param_split_range = [5, 10, 15, 20, 25, 30]

rf_param_grid = [{'randomforestclassifier__criterion': ['gini', 'entropy'],
                  'randomforestclassifier__n_estimators': rf_param_estimators_range,
                  'randomforestclassifier__max_depth': rf_param_depth_range,
                  'randomforestclassifier__min_samples_split': rf_param_split_range}]
rf_gs = GridSearchCV(estimator=pl_randf, param_grid=rf_param_grid, scoring='accuracy', cv=10, refit=True, n_jobs=-1)
rf_gs.fit(X_train, y_train)

print('CV best accuracy:', rf_gs.best_score_)
print(rf_gs.best_params_)
rf_bestclf = rf_gs.best_estimator_
print('Test accuracy: %f' % rf_bestclf.score(X_check, y_check))

#rf_pred = rf_bestclf.predict(X_test)

CV best accuracy: 0.9000011526712391
{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 5, 'randomforestclassifier__min_samples_split': 5, 'randomforestclassifier__n_estimators': 100}
Test accuracy: 0.899995
